## web app

In [12]:
import pandas as pd
import plotly.graph_objects as go

import dash
import dash_table
import dash_core_components as dcc
import dash_html_components as html
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output

from jupyter_dash import JupyterDash
app = JupyterDash("myapp")

# Create server variable with Flask server object for use with gunicorn
#server = app.server

app = dash.Dash(__name__)
server = app.server

target_url = 'https://api.covid19india.org/csv/latest/case_time_series.csv'
case_file = pd.read_csv (target_url)
    
def get_covid_status_graph():
    # Create traces
    fig = go.Figure()
    fig.layout.xaxis.zeroline = False
    fig.layout.yaxis.zeroline = False
    #Total Confirmed cases
    fig.add_trace(go.Scatter(x=case_file.Date, y=case_file["Total Confirmed"],
                    mode='lines',
                    line_color='tomato',
                    name='Total Confirmed'))

    #Total Active cases
    fig.add_trace(go.Scatter(x=case_file.Date, y=case_file["Total Recovered"] + case_file["Total Deceased"],
                    mode='lines',
                    fill='tonexty',
                    name='Total Active',
                    fillcolor = 'thistle',
                    line_color='gold'))

    #Total Recovered cases
    fig.add_trace(go.Scatter(x=case_file.Date, y=case_file["Total Recovered"],
                    mode='lines',
                    name='Total Recovered',
                    line_color='yellowgreen'))


    fig.add_trace(go.Scatter(x=case_file.Date, y=case_file["Total Deceased"],
                    mode='lines',
                    fill='tozeroy',
                    name='Total Deceased',
                    fillcolor = 'lightgray',
                    line_color='rosybrown'))

    fig.add_trace(go.Scatter(
    x=["25 March ", "25 March "],
    y=[case_file["Total Confirmed"].max(),0],
    mode="lines+text",
    name="Lockdown starts",
    text=["Lockdown"],
    textposition="top center",
     hoverinfo='none',
    showlegend=False,
    line_color='darkred'
    ))

    fig.add_trace(go.Scatter(
    x=["01 June ", "01 June "],
    y=[case_file["Total Confirmed"].max(),0],
    mode="lines+text",
    name="Unlock 1.0",
    text=["Unlock 1.0"],
    textposition="top center",
     hoverinfo='none',
    showlegend=False,
    line_color='palegreen'
    ))

    fig.add_trace(go.Scatter(
    x=["01 July ", "01 July "],
    y=[case_file["Total Confirmed"].max(),0],
    mode="lines+text",
    name="Unlock 2.0",
    text=["Unlock 2.0"],
    textposition="top center",
     hoverinfo='none',
    showlegend=False,
    line_color='mediumaquamarine'
    ))
    fig.update_layout(
    title = "Covid Status in India",
    autosize=True,
   # width=800,
   # height=500,
    hovermode="x unified",
    margin=dict(
        l=50,
        r=50,
        b=10,
        t=100,
        pad=10),
    paper_bgcolor="papayawhip",

    )
    
    return fig

def get_death_rate():
    total_days = (len(case_file)*24*60*60)
    total_deaths = case_file['Daily Deceased'].sum()
    death_rate = total_days/total_deaths
    minutes = int(death_rate/60)
    seconds = int(round(death_rate%60,0))
    return 'Someone is dying in every {} minutes and {} seconds in India.'.format(minutes, seconds)

def get_daily_data():
    daily_data = case_file.tail(1)
    return daily_data[["Daily Confirmed","Daily Recovered","Daily Deceased"]]

alerts = html.Div(
    [
        dbc.Alert(get_death_rate(), color="danger"),
    ]
app.layout = html.Div([
    html.Div(html.Div(children = get_death_rate(),id='death_rate'),
             style={'color': 'red','font-size': '20px','margin-bottom': '50px',
                    'margin-top': '25px', 'width': 'auto','text-align': 'center',
                    'background-color': 'aliceblue'}),
    
    html.Div([
        html.H3("Today's Status"),
        dash_table.DataTable(
        id='table',
        columns=[{"name": i, "id": i} for i in get_daily_data().columns],
        data=get_daily_data().to_dict("records"),
        style_header={'backgroundColor': 'rgb(30, 30, 30)'},
        style_cell={
        'backgroundColor': 'rgb(50, 50, 50)',
        'color': 'white',
        'textAlign': 'left'
    })
        
    ],style={'margin-bottom': '50px','margin-top': '25px','width':'50%'},),
    html.Div([
        dcc.Graph(figure = get_covid_status_graph())
    ],style={'margin-bottom': '50px','margin-top': '25px', 'width':'50%', 'height':'500'})
])

app.run_server()
#app.run_server(mode="inline")
#if __name__ == '__main__':
    #app.run_server(debug=True)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [24/Jul/2020 09:08:18] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [24/Jul/2020 09:08:18] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [24/Jul/2020 09:08:18] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [24/Jul/2020 09:08:18] "GET /_favicon.ico HTTP/1.1" 200 -


## death rate

In [22]:
total_days = (len(case_file)*24*60*60)
total_deaths = case_file['Daily Deceased'].sum()
death_rate = total_days/total_deaths
minutes = int(death_rate/60)
seconds = int(round(death_rate%60,0))
print('Someone is dying in every {} minutes and {} seconds in India.'.format(minutes, seconds))

Someone is dying in every 8 minutes and 26 seconds in India.


## death graph

In [33]:
case_file.tail(1)[["Daily Confirmed","Daily Recovered","Daily Deceased"]]

,Daily Confirmed,Daily Recovered,Daily Deceased
174,45601,31875,1130


In [18]:
(death_rate%60)
death_rate%60

25.651795866497253

In [37]:
def get_daily_data():
    daily_data = case_file.tail(1)
    return daily_data[["Daily Confirmed","Daily Recovered","Daily Deceased"]]

for i in get_daily_data().columns:
    print(i)

Daily Confirmed
Daily Recovered
Daily Deceased


In [39]:
get_daily_data().to_dict("records")

[{'Daily Confirmed': 45601, 'Daily Recovered': 31875, 'Daily Deceased': 1130}]

## daily test vs daily confirmed - state wise

In [44]:
import pandas as pd
import plotly.graph_objects as go
test_file_location = "https://api.covid19india.org/csv/latest/statewise_tested_numbers_data.csv"
test_data = pd.read_csv(test_file_location)
test_data.head(2)

,Updated On,State,Total Tested,Tag (Total Tested),Positive,Negative,Unconfirmed,Cumulative People In Quarantine,Total People Currently in Quarantine,Tag (People in Quarantine),...,Corona Enquiry Calls,Num Calls State Helpline,Source1,Source2,Source3,Test positivity rate,Tests per thousand,Tests per million,Tests per positive case,Population NCP 2019 Projection
0,17/04/2020,Andaman and Nicobar Islands,1403.0,Samples Sent,12.0,1210,181.0,NaN,NaN,NaN,...,NaN,NaN,https://t.me/indiacovid/2550,NaN,NaN,0.86%,3.53,3534.0,117,397000.0
1,24/04/2020,Andaman and Nicobar Islands,2679.0,Samples Sent,27.0,NaN,246.0,NaN,614.0,Institutional,...,NaN,280.0,https://t.me/indiacovid/3147?single,NaN,NaN,1.01%,6.75,6748.0,99,397000.0


In [34]:
from datetime import date 
from datetime import datetime
from datetime import timedelta
#check delta
last_day_1 = (date.today()  - timedelta(days = 1)).strftime('%d/%m/%Y') 
last_day_2 = (date.today()  - timedelta(days = 2)).strftime('%d/%m/%Y') 

#yesterday = yesterday.strftime("%x").replace('/','-')
last_day_1

'28/07/2020'

In [46]:

filtered_data_1 = test_data[test_data['Updated On'] == last_day_1]
filtered_data_2 = test_data[test_data['Updated On'] == last_day_2]
filtered_data_2["Total Tested"]

71        21743.0
179     1541993.0
278       53335.0
372      739465.0
483      429664.0
591       12030.0
694      274660.0
785           NaN
895      908735.0
999      119455.0
1108     606718.0
1220     505220.0
1328     127555.0
1437     559399.0
1543     248928.0
1655    1143262.0
1771     635272.0
1841      17395.0
1953     670155.0
2064    1790610.0
2128      75932.0
2216      31367.0
2322      19238.0
2431      32909.0
2543     433578.0
2643      34305.0
2754     509267.0
2866    1349544.0
2947      20785.0
3058    2223019.0
3102     337771.0
3194     131485.0
3305    1705348.0
3416     135447.0
3529     773512.0
Name: Total Tested, dtype: float64

In [47]:
state_list = []
no_of_tests = []
positive_cases = []
positivity_rate = []
df_columns = filtered_data_1.values
counter = 0
for index, row in filtered_data_1.iterrows():
    state_list.append(row["State"])
    no_of_tests.append((row["Total Tested"] - filtered_data_2[filtered_data_2['State'] == row['State']]["Total Tested"].values[0]))
    positive_cases.append(row["Positive"] - filtered_data_2[filtered_data_2['State'] == row['State']]["Positive"].values[0])
    positivity_rate.append((positive_cases[counter]/no_of_tests[counter])*100)
    counter += 1


In [48]:
filtered_data_1[filtered_data_1['State'] == "Karnataka"]["Total Tested"].values[0] - filtered_data_2[filtered_data_2['State'] == "Karnataka"]["Total Tested"].values[0]

filtered_data_2[filtered_data_2['State'] == "Karnataka"]["Total Tested"].values[0]

1143262.0

In [49]:
graph_data = {'State': state_list,
        'Total Tested': no_of_tests,
        'Positive': positive_cases,
        'Positivity Rate': positivity_rate
        }

df = pd.DataFrame (graph_data, columns = ['State','Total Tested','Positive','Positivity Rate'])
df = df.sort_values(by=['Total Tested'])
df

,State,Total Tested,Positive,Positivity Rate
17,Ladakh,230.0,30.0,13.043478
5,Chandigarh,328.0,29.0,8.841463
28,Sikkim,487.0,35.0,7.186858
22,Mizoram,543.0,29.0,5.340700
25,Puducherry,775.0,139.0,17.935484
23,Nagaland,795.0,51.0,6.415094
21,Meghalaya,1034.0,13.0,1.257253
20,Manipur,1060.0,30.0,2.830189
9,Goa,2316.0,146.0,6.303972
12,Himachal Pradesh,2698.0,95.0,3.521127


In [2]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(go.Bar(
    x=df['State'], 
    y=df['Total Tested'],
    name="Daily Tests"
), secondary_y=False)

fig.add_trace(go.Bar(
    x=df['State'], 
    y=df['Positive'],
    name="Positive Cases",
    marker={'color': 'yellow'}
), secondary_y=False)


# Add traces
fig.add_trace(go.Scatter(
x=df['State'],
y=df['Positivity Rate'],
mode="lines",
name="Positivity Rate",
line_color='tomato'
), secondary_y=True)

# Change the bar mode
fig.update_layout(barmode='stack',title = "Daily Tests vs Positive Cases",
    autosize=True,
   # width=800,
   # height=500
    margin=dict(
        l=50,
        r=50,
        t=100,
        pad=10),
     hovermode="x unified"
)
fig.show()

NameError: name 'df' is not defined

## daily test - positivity rate - India

In [8]:
import pandas as pd
import plotly.graph_objects as go
daily_test_url = 'https://api.covid19india.org/csv/latest/tested_numbers_icmr_data.csv'
daily_test_data = pd.read_csv (daily_test_url)
sample_daily_test_data = daily_test_data.tail(14)
target_url = 'https://api.covid19india.org/csv/latest/case_time_series.csv'
case_file = pd.read_csv (target_url)
sample_case_data = case_file.tail(14)

#case_file.tail(14)


In [30]:

case_file.tail(2)


,Date,Daily Confirmed,Total Confirmed,Daily Recovered,Total Recovered,Daily Deceased,Total Deceased
177,25 July,50072,1387088,37125,887124,703,32123
178,26 July,48931,1436019,31501,918625,702,32825


In [15]:
#(sample_case_data.values[0][1]/int(sample_daily_test_data.values[0][6]))*100


13.284163156141176

In [62]:
positivity_rate = []
no_of_tests = []
counter = 0
for index, row in sample_case_data.iterrows():
    no_of_tests.append(sample_daily_test_data.values[counter][6])
    positivity_rate.append(round((sample_case_data.values[counter][1]/int(sample_daily_test_data.values[counter][6])*100),2))
    counter += 1
positivity_rate

[12.86,
 10.45,
 10.18,
 10.85,
 10.45,
 10.36,
 11.23,
 14.38,
 11.75,
 13.29,
 13.81,
 13.86,
 11.9,
 11.06]

In [63]:
#sample_daily_test_data

In [64]:
graph_data = {'Date': sample_case_data['Date'],
        'Number Of Tests': no_of_tests,
        'Positive': sample_case_data['Daily Confirmed'],
        'Positivity Rate': positivity_rate
        }

df = pd.DataFrame (graph_data, columns = ['Date','Number Of Tests','Positive','Positivity Rate'])
df

,Date,Number Of Tests,Positive,Positivity Rate
165,13 July,219103,28178,12.86
166,14 July,286247,29917,10.45
167,15 July,320161,32607,10.18
168,16 July,326826,35468,10.85
169,17 July,333228,34820,10.45
170,18 July,361024,37411,10.36
171,19 July,358127,40235,11.23
172,20 July,256039,36806,14.38
173,21 July,333395,39170,11.75
174,22 July,343243,45601,13.29


In [65]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(go.Bar(
    x=df["Date"], 
    y=df['Positive'],
    name="Positive Cases",
    marker={'color': 'yellow'}
), secondary_y=False)

fig.add_trace(go.Bar(
    x=df["Date"], 
    y=df['Number Of Tests'],
    name="Number of tests",
    marker={'color': 'blue'}
), secondary_y=False)


# Add traces
fig.add_trace(go.Scatter(
x=df['Date'],
y=df['Positivity Rate'],
mode="lines",
name="Positivity Rate (in %)",
line_color='tomato'
), secondary_y=True)

# Change the bar mode
fig.update_layout(barmode='stack',title = "Daily Tests vs Positive Cases",
    autosize=True,
   # width=800,
   # height=500
    margin=dict(
        l=50,
        r=50,
        t=100,
        pad=10),
     hovermode="x unified",
    )
fig.show()

## death rate graph - state wise


In [53]:
import pandas as pd
import plotly.graph_objects as go
state_wise_daily_url = 'https://api.covid19india.org/csv/latest/state_wise_daily.csv'
state_wise_daily_data = pd.read_csv (state_wise_daily_url)
state_wise_daily_data.head(3)

,Date,Status,TT,AN,AP,AR,AS,BR,CH,CT,...,PB,RJ,SK,TN,TG,TR,UP,UT,WB,UN
0,14-Mar-20,Confirmed,81,0,1,0,0,0,0,0,...,1,3,0,1,1,0,12,0,0,0
1,14-Mar-20,Recovered,9,0,0,0,0,0,0,0,...,0,1,0,0,0,0,4,0,0,0
2,14-Mar-20,Deceased,2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [54]:

#state_wise_daily_data.head(2)

In [55]:
#type(state_wise_daily_data.values[2])

In [56]:
state_wise_deceased_data = state_wise_daily_data[state_wise_daily_data['Status'] == 'Deceased']
state_wise_deceased_data = state_wise_deceased_data.drop(['TT','DD'], axis=1)
state_wise_deceased_data = state_wise_deceased_data.reset_index()

state_wise_confirmed_data = state_wise_daily_data[state_wise_daily_data['Status'] == 'Confirmed']
state_wise_confirmed_data = state_wise_confirmed_data.drop(['TT','DD'], axis=1)
state_wise_confirmed_data = state_wise_confirmed_data.reset_index()

state_code = list(state_wise_daily_data.columns[2:])
#state_code

In [26]:
#(state_wise_deceased_data['TN']/state_wise_confirmed_data['TN']).tail(30).values[1]
state_wise_deceased_data.cumsum()['TN']


0         0
1         0
2         0
3         0
4         0
       ... 
132    3320
133    3409
134    3494
135    3571
136    3659
Name: TN, Length: 137, dtype: int64

In [4]:
state_wise_url = 'https://api.covid19india.org/csv/latest/state_wise.csv'
state_wise_data = pd.read_csv(state_wise_url)

#state_code_list[state_code_list['State_code']=='TT']['State'].values[0]

In [81]:
state_code_list = state_wise_data[['State','State_code']][1:]
state_code_list= state_code_list.sort_values(by=['State_code'])
#state_code_list

In [82]:
state_wise_confirmed_data.loc[0:28].sum()["HP"]

32

In [83]:
state_wise_deceased_data.loc[0:28].sum()["HP"]

2

In [84]:
#total_deceased_cases_biweekly/total_confirmed_cases_biweekly

In [85]:
import math
#number of biweekly chunks required
n = int(state_wise_deceased_data.shape[0]/14)

In [86]:
#state_wise_confirmed_data.loc[lower_limit:upper_limit].sum()

In [87]:
data = {'Days Interval': [0]
       }
data_set = pd.DataFrame(data) 
for i in state_code_list['State_code']:
    data_set[i] = 'Nan'
data_set

,Days Interval,AN,AP,AR,AS,BR,CH,CT,DL,DN,...,PY,RJ,SK,TG,TN,TR,UN,UP,UT,WB
0,0,Nan,Nan,Nan,Nan,Nan,Nan,Nan,Nan,Nan,...,Nan,Nan,Nan,Nan,Nan,Nan,Nan,Nan,Nan,Nan


In [88]:
# a period of 2 weeks
lower_limit = 0
upper_limit = 13
state_wise_death_rate = {}
for i in range(n):
    row = [upper_limit + 1]
    total_confirmed_cases_biweekly = state_wise_confirmed_data.loc[lower_limit:upper_limit].sum()
    total_deceased_cases_biweekly = state_wise_deceased_data.loc[lower_limit:upper_limit].sum()
    for i in state_code_list['State_code']:
        if total_confirmed_cases_biweekly[i] == 0:
            state_wise_death_rate[i] = 0
        else:
            state_wise_death_rate[i] = round((total_deceased_cases_biweekly[i]/total_confirmed_cases_biweekly[i])*100,2)
        row.append(state_wise_death_rate[i])
    data_set.loc[len(data_set)] = row
    lower_limit =upper_limit + 1
    upper_limit += 14
row = [upper_limit + 1]
row += ['Nan']*37
data_set.loc[len(data_set)] = row
data_set

,Days Interval,AN,AP,AR,AS,BR,CH,CT,DL,DN,...,PY,RJ,SK,TG,TN,TR,UN,UP,UT,WB
0,0.0,Nan,Nan,Nan,Nan,Nan,Nan,Nan,Nan,Nan,...,Nan,Nan,Nan,Nan,Nan,Nan,Nan,Nan,Nan,Nan
1,14.0,0,0,0,0,11.11,0,0,2.5,0,...,0,0,0,0,2.63,0,0,0,0,6.67
2,28.0,0,1.63,0,3.45,0,0,0,1.51,0,...,0,0.59,0,2.8,0.92,0,0,1.04,0,3.96
3,42.0,0,4.01,0,0,0.61,0,0,2.42,0,...,0,1.97,0,2.62,1.54,0,0,1.77,0,3.27
4,56.0,0,1.29,0,4.17,0.84,1.68,0,0.39,0,...,0,4.6,0,2.68,0.42,0,0,2.57,6.67,12.2
5,70.0,0,1.79,0,0.6,0.38,1.37,0,2.33,0,...,0,1.72,0,3.02,0.67,0,0,3.41,0,6.35
6,84.0,0,1.14,0,0,0.74,2.22,0.29,3.57,0,...,0,1.81,0,4.25,0.98,0,0,2.63,0.94,2.54
7,98.0,0,0.62,0,0.19,0.74,1.39,0.69,4.95,0,...,3.83,2.82,0,2.63,1.67,0.21,0,3.64,1.56,2.82
8,112.0,0,1.23,0.85,0.1,0.92,0,0.38,2.14,0,...,1.12,2.19,0,0.61,1.49,0,-0,2.63,1.84,2.54
9,126.0,0,1.38,0.56,0.35,0.73,2.43,0.52,2.55,0.66,...,1.19,1.21,0,0.54,1.6,0.21,-0,1.73,0.85,1.89


In [89]:
data = []
for i in state_code_list['State_code']:
    data.append(go.Scatter(x=data_set['Days Interval'], y=data_set[i], mode="lines",
                           name=state_code_list[state_code_list['State_code']==i]['State'].values[0]))

fig = go.Figure(data=data)

# Suffix y-axis tick labels with % sign
fig.update_yaxes(ticksuffix="%")
fig.update_xaxes(tick0=0, dtick=14)

fig.update_layout(
    title={
        'text': "Death Rate - State wise overview",
        'y':0.9,
        'x':0.3,
        'xanchor': 'center',
        'yanchor': 'top'},

    xaxis_title="No Of Days",
    yaxis_title="Death rate",
    legend_title="States",
    hovermode = "x",
    font=dict(
        family="Courier New, monospace",
        size=15,
        color="RebeccaPurple"
    )
)


fig.show()

## tpm vs cpm graph

In [11]:
import pandas as pd
import plotly.graph_objects as go 
from plotly.subplots import make_subplots

test_file_location = "https://api.covid19india.org/csv/latest/statewise_tested_numbers_data.csv"
test_data = pd.read_csv(test_file_location)
state_wise_url = 'https://api.covid19india.org/csv/latest/state_wise.csv'
state_wise_data = pd.read_csv(state_wise_url)


In [56]:
test_data['Case Per Million'] = round((test_data['Positive']/(test_data['Population NCP 2019 Projection']/1000000)))
test_data.head(5)

,Updated On,State,Total Tested,Tag (Total Tested),Positive,Negative,Unconfirmed,Cumulative People In Quarantine,Total People Currently in Quarantine,Tag (People in Quarantine),...,Source1,Source2,Source3,Test positivity rate,Tests per thousand,Tests per million,Tests per positive case,Population NCP 2019 Projection,Case Per Test,Case Per Million
0,17/04/2020,Andaman and Nicobar Islands,1403.0,Samples Sent,12.0,1210,181.0,NaN,NaN,NaN,...,https://t.me/indiacovid/2550,NaN,NaN,0.86%,3.53,3534.0,117,397000.0,30.22670,30.0
1,24/04/2020,Andaman and Nicobar Islands,2679.0,Samples Sent,27.0,NaN,246.0,NaN,614.0,Institutional,...,https://t.me/indiacovid/3147?single,NaN,NaN,1.01%,6.75,6748.0,99,397000.0,68.01008,68.0
2,27/04/2020,Andaman and Nicobar Islands,2848.0,Samples Sent,33.0,NaN,106.0,NaN,724.0,Institutional,...,https://t.me/indiacovid/3365?single,NaN,NaN,1.16%,7.17,7174.0,86,397000.0,83.12343,83.0
3,01/05/2020,Andaman and Nicobar Islands,3754.0,Samples Sent,33.0,NaN,199.0,NaN,643.0,Institutional,...,https://t.me/indiacovid/3781,NaN,NaN,0.88%,9.46,9456.0,114,397000.0,83.12343,83.0
4,16/05/2020,Andaman and Nicobar Islands,6677.0,Samples Sent,33.0,NaN,136.0,NaN,16.0,Institutional,...,https://t.me/indiacovid/4925,NaN,NaN,0.49%,16.82,16819.0,202,397000.0,83.12343,83.0


In [57]:
group_data = test_data.groupby('State')
group_data.get_group('Delhi')['Case Per Million']
#group_data.groups.keys()

805       NaN
806      29.0
807      36.0
808      46.0
809      54.0
        ...  
914    6537.0
915    6592.0
916    6623.0
917    6676.0
918       NaN
Name: Case Per Million, Length: 114, dtype: float64

In [2]:
state_code_list = state_wise_data[['State','State_code']][1:]
state_code_list= state_code_list.sort_values(by=['State_code'])

In [59]:
data = []
for i in group_data.groups.keys():
    
    data.append(go.Scatter(x=group_data.get_group(i)['Case Per Million'], 
                           y=group_data.get_group(i)['Tests per million'], mode="lines",
                           name=i))

fig = go.Figure(data=data)


fig.update_layout(
    title={
        'text': "Test Per Million v/s Case Per Million",
        'y':0.9,
        'x':0.3,
        'xanchor': 'center',
        'yanchor': 'top'},

    xaxis_title="Case Per Million",
    yaxis_title="Test per million",
    legend_title="States",
    font=dict(
        family="Courier New, monospace",
        size=15,
        color="RebeccaPurple"
    )
)


fig.show()

## case fatality rate vs test per confirmed cases

In [44]:
last_updated_date = state_wise_daily_data.tail(1)['Date']
last_updated_date.values[0]

'28-Jul-20'

In [45]:
filtered_data = test_data[test_data['Updated On'] >= last_updated_date][['State','Tests per positive case']]

ValueError: Can only compare identically-labeled Series objects

In [46]:
test_per_cases_data = filtered_data.groupby('State')
test_per_cases_data.get_group('Kerala')

,State,Tests per positive case
1702,Kerala,30
1703,Kerala,30
1704,Kerala,31
1705,Kerala,32
1706,Kerala,33
...,...,...
1816,Kerala,37
1817,Kerala,36
1818,Kerala,35
1819,Kerala,35


In [47]:

from datetime import datetime
date_time_obj = datetime. strptime(last_updated_date.values[0], '%d-%b-%y')
last_updated_date = (date_time_obj + timedelta(days = 1)).strftime('%d-%b-%y')
last_updated_date

'29-Jul-20'

In [48]:

filtered_data = test_data[test_data['Updated On'] < last_updated_date][['State','Tests per positive case']]
test_per_cases_data = filtered_data.groupby('State')
test_per_cases_data.get_group('Kerala').tail(30)

,State,Tests per positive case
1789,Kerala,53
1790,Kerala,53
1793,Kerala,52
1794,Kerala,52
1795,Kerala,51
1796,Kerala,50
1797,Kerala,49
1798,Kerala,49
1799,Kerala,49
1800,Kerala,48


In [49]:
state_code_list = state_wise_data[['State','State_code']][1:]
state_code_list= state_code_list.sort_values(by=['State_code'])

In [57]:
fatality_rate = ((state_wise_deceased_data['TN']/state_wise_confirmed_data['TN'])*100).tail(30)
#fatality_rate

In [58]:
data = []
cum_deceased_data = state_wise_deceased_data.cumsum()
cum_confirmed_data = state_wise_confirmed_data.cumsum()
for i in test_per_cases_data.groups.keys():
    test_per_confirmed = test_per_cases_data.get_group(i).tail(30)
    state_code = state_code_list[state_code_list['State']==i]['State_code'].values[0]
    fatality_rate = ((cum_deceased_data[state_code]/cum_confirmed_data[state_code])*100).tail(30)
    data.append(go.Scatter(x=test_per_confirmed['Tests per positive case'], 
                       y=fatality_rate, mode="lines",
                       name=i))

In [59]:


fig = go.Figure(data=data)
fig.update_yaxes(ticksuffix="%")

fig.update_layout(
    title={
        'text': "Case Fatality Rate v/s Test Per Positive Case",
        'y':0.9,
        'x':0.3,
        'xanchor': 'center',
        'yanchor': 'top'},

    xaxis_title="Test per positive case",
    yaxis_title="Case fatality rate",
    legend_title="States",

)



## notebook app


In [60]:
import pandas as pd
import plotly.graph_objects as go 
from plotly.subplots import make_subplots

import dash
import dash_table
import dash_core_components as dcc
import dash_html_components as html
import dash_bootstrap_components as dbc
import dash_html_components as html

from datetime import date 
from datetime import datetime
from datetime import timedelta

import math

from jupyter_dash import JupyterDash
app = JupyterDash("myapp")

app = dash.Dash(external_stylesheets=[dbc.themes.BOOTSTRAP])


target_url = 'https://api.covid19india.org/csv/latest/case_time_series.csv'
case_file = pd.read_csv (target_url)
test_file_location = "https://api.covid19india.org/csv/latest/statewise_tested_numbers_data.csv"
test_data = pd.read_csv(test_file_location)
state_wise_daily_url = 'https://api.covid19india.org/csv/latest/state_wise_daily.csv'
state_wise_daily_data = pd.read_csv (state_wise_daily_url)


In [61]:
def get_death_rate():
    total_days = (len(case_file)*24*60*60)
    total_deaths = case_file['Daily Deceased'].sum()
    death_rate = total_days/total_deaths
    minutes = int(death_rate/60)
    seconds = int(round(death_rate%60,0))
    return 'Someone is dying in every {} minutes and {} seconds in India!'.format(minutes, seconds)

def get_covid_status_graph():
    # Create traces
    fig = go.Figure()
    fig.layout.xaxis.zeroline = False
    fig.layout.yaxis.zeroline = False
    #Total Confirmed cases
    fig.add_trace(go.Scatter(x=case_file.Date, y=case_file["Total Confirmed"],
                    mode='lines',
                    line_color='tomato',
                    name='Total Confirmed'))

    #Total Active cases
    fig.add_trace(go.Scatter(x=case_file.Date, y=case_file["Total Recovered"] + case_file["Total Deceased"],
                    mode='lines',
                    fill='tonexty',
                    name='Total Active',
                    fillcolor = 'thistle',
                    line_color='gold'))

    #Total Recovered cases
    fig.add_trace(go.Scatter(x=case_file.Date, y=case_file["Total Recovered"],
                    mode='lines',
                    name='Total Recovered',
                    line_color='yellowgreen'))


    fig.add_trace(go.Scatter(x=case_file.Date, y=case_file["Total Deceased"],
                    mode='lines',
                    fill='tozeroy',
                    name='Total Deceased',
                    fillcolor = 'lightgray',
                    line_color='rosybrown'))

    fig.add_trace(go.Scatter(
    x=["25 March ", "25 March "],
    y=[case_file["Total Confirmed"].max(),0],
    mode="lines+text",
    name="Lockdown starts",
    text=["Lockdown"],
    textposition="top center",
     hoverinfo='none',
    showlegend=False,
    line_color='darkred'
    ))

    fig.add_trace(go.Scatter(
    x=["01 June ", "01 June "],
    y=[case_file["Total Confirmed"].max(),0],
    mode="lines+text",
    name="Unlock 1.0",
    text=["Unlock 1.0"],
    textposition="top center",
     hoverinfo='none',
    showlegend=False,
    line_color='palegreen'
    ))

    fig.add_trace(go.Scatter(
    x=["01 July ", "01 July "],
    y=[case_file["Total Confirmed"].max(),0],
    mode="lines+text",
    name="Unlock 2.0",
    text=["Unlock 2.0"],
    textposition="top center",
     hoverinfo='none',
    showlegend=False,
    line_color='mediumaquamarine'
    ))
    fig.update_layout(
    title = "Covid Status in India",
    autosize=True,
   # width=800,
   # height=500,
    hovermode="x unified",
    margin=dict(
        l=50,
        r=50,
        b=10,
        t=100,
        pad=10),

    )
    
    return fig


def get_daily_data():
    daily_data = case_file.tail(1)
    return daily_data[["Daily Confirmed","Daily Recovered","Daily Deceased"]]

def get_status_cards():
    card_content_confirmed = [
        dbc.CardHeader("Confirmed"),
        dbc.CardBody(
            [
                html.P(
                    get_daily_data()["Daily Confirmed"].values[0],
                    className="card-text",
                ),
            ]
        ),
    ]
    card_content_recovered = [
        dbc.CardHeader("Recovered"),
        dbc.CardBody(
            [
                html.P(
                    get_daily_data()["Daily Recovered"].values[0],
                    className="card-text",
                ),
            ]
        ),
    ]
    card_content_deceased = [
        dbc.CardHeader("Deceased"),
        dbc.CardBody(
            [
                html.P(
                    get_daily_data()["Daily Deceased"].values[0],
                    className="card-text",
                ),
            ]
        ),
    ]
    status_cards = dbc.Row(
        [
            dbc.Col(dbc.Card(card_content_confirmed, color="danger", inverse=True)),
            dbc.Col(dbc.Card(card_content_recovered, color="success",  inverse=True)),
            dbc.Col(dbc.Card(card_content_deceased, color="secondary",  inverse=True)),
        ],
        className="mb-4",
    )
    return status_cards



In [62]:
def get_tests_vs_positive_data():
    daily_test_url = 'https://api.covid19india.org/csv/latest/tested_numbers_icmr_data.csv'
    daily_test_data = pd.read_csv (daily_test_url)
    sample_daily_test_data = daily_test_data.tail(14)
    sample_case_data = case_file.tail(14)
    positivity_rate = []
    no_of_tests = []
    counter = 0
    for index, row in sample_case_data.iterrows():
        no_of_tests.append(sample_daily_test_data.values[counter][6])
        positivity_rate.append(round((sample_case_data.values[counter][1]/int(sample_daily_test_data.values[counter][6])*100),2))
        counter += 1
    graph_data = {'Date': sample_case_data['Date'],
        'Number Of Tests': no_of_tests,
        'Positive': sample_case_data['Daily Confirmed'],
        'Positivity Rate': positivity_rate
        }

    df = pd.DataFrame (graph_data, columns = ['Date','Number Of Tests','Positive','Positivity Rate'])
    return df

def get_tests_vs_positive_graph():
   # Create figure with secondary y-axis
    fig = make_subplots(specs=[[{"secondary_y": True}]])
    df = get_tests_vs_positive_data()
    fig.add_trace(go.Bar(
        x=df["Date"], 
        y=df['Positive'],
        name="Positive Cases",
        marker={'color': 'yellow'}
    ), secondary_y=False)

    fig.add_trace(go.Bar(
        x=df["Date"], 
        y=df['Number Of Tests'],
        name="Number of tests",
        marker={'color': 'blue'}
    ), secondary_y=False)


    # Add traces
    fig.add_trace(go.Scatter(
    x=df['Date'],
    y=df['Positivity Rate'],
    mode="lines",
    name="Positivity Rate (in %)",
    line_color='tomato'
    ), secondary_y=True)

    # Change the bar mode
    fig.update_layout(barmode='stack',title = "Daily Tests vs Positive Cases",
        autosize=True,
       # width=800,
       # height=500
        margin=dict(
            l=50,
            r=50,
            t=100,
            pad=10),
         hovermode="x unified",
        )
    return fig

In [63]:
def get_death_data():
    state_wise_deceased_data = state_wise_daily_data[state_wise_daily_data['Status'] == 'Deceased']
    state_wise_deceased_data = state_wise_deceased_data.drop(['TT','DD'], axis=1)
    state_wise_deceased_data = state_wise_deceased_data.reset_index()
    return state_wise_deceased_data

def get_confirmed_data():
    state_wise_confirmed_data = state_wise_daily_data[state_wise_daily_data['Status'] == 'Confirmed']
    state_wise_confirmed_data = state_wise_confirmed_data.drop(['TT','DD'], axis=1)
    state_wise_confirmed_data = state_wise_confirmed_data.reset_index()
    return state_wise_confirmed_data
    
def get_state_code_list():
    state_wise_url = 'https://api.covid19india.org/csv/latest/state_wise.csv'
    state_wise_data = pd.read_csv(state_wise_url)
    state_code_list = state_wise_data[['State','State_code']][1:]
    return state_code_list

def get_death_rate_data():
    state_wise_deceased_data = get_death_data()
    state_wise_confirmed_data = get_confirmed_data()
    #number of biweekly chunks required
    n = int(state_wise_deceased_data.shape[0]/14)
    data = {'Days Interval': [0]
       }
    data_set = pd.DataFrame(data) 
    state_code_list = get_state_code_list()
    for i in state_code_list['State_code']:
        data_set[i] = 'Nan'
    # a period of 2 weeks
    lower_limit = 0
    upper_limit = 13
    state_wise_death_rate = {}
    for i in range(n+1):
        row = [upper_limit + 1]
        total_confirmed_cases_biweekly = state_wise_confirmed_data.loc[lower_limit:upper_limit].sum()
        total_deceased_cases_biweekly = state_wise_deceased_data.loc[lower_limit:upper_limit].sum()
        for i in state_code_list['State_code']:
            if total_confirmed_cases_biweekly[i] == 0:
                state_wise_death_rate[i] = 0
            else:
                state_wise_death_rate[i] = round((total_deceased_cases_biweekly[i]/total_confirmed_cases_biweekly[i])*100,2)
            row.append(state_wise_death_rate[i])
        data_set.loc[len(data_set)] = row
        lower_limit =upper_limit + 1
        upper_limit += 14
    row = [upper_limit + 1]
    row += ['Nan']*37
    data_set.loc[len(data_set)] = row
    return data_set

def get_death_rate_graph():
    data = []
    state_code_list = get_state_code_list()
    data_set = get_death_rate_data()
    for i in state_code_list['State_code']:
        data.append(go.Scatter(x=data_set['Days Interval'], y=data_set[i], mode="lines",
                               name=state_code_list[state_code_list['State_code']==i]['State'].values[0]))

    fig = go.Figure(data=data)

    # Suffix y-axis tick labels with % sign
    fig.update_yaxes(ticksuffix="%")
    fig.update_xaxes(tick0=0, dtick=14)

    fig.update_layout(
        title={
            'text': "Death Rate - State wise overview",
            'y':0.9,
            'x':0.3,
            'xanchor': 'center',
            'yanchor': 'top'},

        xaxis_title="No Of Days",
        yaxis_title="Death rate",
        legend_title="States",
        hovermode = "x",
        font=dict(
            family="Courier New, monospace",
            size=15,
            color="RebeccaPurple"
        )
    )

    return fig

In [64]:
def get_tpm_cpm_data():
    test_data['Case Per Million'] = round((test_data['Positive']/(test_data['Population NCP 2019 Projection']/1000000)))
    group_data = test_data.groupby('State')
    return group_data

def get_tpm_cpm_graph():
    data = []
    group_data = get_tpm_cpm_data()
    for i in group_data.groups.keys():

        data.append(go.Scatter(x=group_data.get_group(i)['Case Per Million'], 
                               y=group_data.get_group(i)['Tests per million'], mode="lines",
                               name=i))

    fig = go.Figure(data=data)


    fig.update_layout(
        title={
            'text': "Test Per Million v/s Case Per Million",
            'y':0.9,
            'x':0.3,
            'xanchor': 'center',
            'yanchor': 'top'},

        xaxis_title="Case Per Million",
        yaxis_title="Test per million",
        legend_title="States",
        font=dict(
            family="Courier New, monospace",
            size=15,
            color="RebeccaPurple"
        )
    )
    
    return fig


In [71]:
def test_per_positive_data():
    last_updated_date = state_wise_daily_data.tail(1)['Date']
    date_time_obj = datetime. strptime(last_updated_date.values[0], '%d-%b-%y')
    last_updated_date = (date_time_obj + timedelta(days = 1)).strftime('%d-%b-%y')
    filtered_data = test_data[test_data['Updated On'] <= last_updated_date][['State','Tests per positive case']]
    test_per_positive_data = filtered_data.groupby('State')
    return test_per_positive_data

def get_fatality_test_per_positive_graph():
    data = []
    cum_deceased_data = state_wise_deceased_data.cumsum()
    cum_confirmed_data = state_wise_confirmed_data.cumsum()
    test_per_positive_data = test_per_positive_data()
    state_code_list = get_state_code_list()
    for i in test_per_cases_data.groups.keys():
        test_per_confirmed = test_per_positive_data.get_group(i).tail(30)
        state_code = state_code_list[state_code_list['State']==i]['State_code'].values[0]
        fatality_rate = ((cum_deceased_data[state_code]/cum_confirmed_data[state_code])*100).tail(30)
        data.append(go.Scatter(x=test_per_confirmed['Tests per positive case'], 
                           y=fatality_rate, mode="lines",
                           name=i))

    fig = go.Figure(data=data)
    fig.update_yaxes(ticksuffix="%")

    fig.update_layout(
        title={
            'text': "Case Fatality Rate v/s Test Per Positive Case",
            'y':0.9,
            'x':0.3,
            'xanchor': 'center',
            'yanchor': 'top'},

        xaxis_title="Test per positive case",
        yaxis_title="Case fatality rate",
        legend_title="States",

    )
    return fig

In [72]:


app.layout = dbc.Container(children=[
    dbc.Alert(get_death_rate(), color="danger",style={'text-align':'center','width':'100'},
    className="p-5"),
    html.Div(get_status_cards()),
    html.Div([dbc.Row(
        [
            dbc.Col(dcc.Graph(figure = get_covid_status_graph(),
                   config={'displayModeBar': False}),)
        ],
        className="mb-4",
    ),
             dbc.Row(
        [
            dbc.Col(dcc.Graph(figure = get_tests_vs_positive_graph(),
                              config={'displayModeBar': False}),),
        ],
        className="mb-4",
    ),
                    dbc.Row(
        [
            dbc.Col(dcc.Graph(figure = get_death_rate_graph(),
                              config={'displayModeBar': False}),),
        ],
        className="mb-4",
    ),
              
                    dbc.Row(
        [
            dbc.Col(dcc.Graph(figure = get_tpm_cpm_graph(),
                              config={'displayModeBar': False}),),
        ],
        className="mb-4",
    ),
               dbc.Row(
        [
            dbc.Col(dcc.Graph(figure = get_fatality_test_per_positive_graph(),
                              config={'displayModeBar': False}),),
        ],
        className="mb-4",
    )
             ])
]
)
app.run_server()

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 in production, use a production WSGI server like gunicorn instead.

 in production, use a production WSGI server like gunicorn instead.

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [29/Jul/2020 10:32:56] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [29/Jul/2020 10:32:57] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [29/Jul/2020 10:32:57] "GET /_dash-layout HTTP/1.1" 200 -
